# Code 5-16 to Code 5-37

For this exercise we are going to take a dialog dataset “Lets Go” bus dataset. The details of the dataset can be found https://dialrc.github.io/LetsGoDataset/ (Raux et al. 2005)Let’s Go was funded by the National Science Foundation. Have cleaned the dataset to represent a conversation format as show in "Clean_lets_go.ipynb"

In [2]:
import pandas as pd
t2 = pd.read_csv("letsgo_op.csv")

In [3]:
t2.head()

,Unnamed: 0,user_id,bot_text,cust_text
0,0,2061123000,welcome to the cmu let's go bus information s...,penn at butler time is it is the centre at hig...
1,1,2061123000,leaving from oakland . is this correct?,oakland
2,2,2061123000,leaving from oakland . is this correct?,yes
3,3,2061123000,leaving from oakland . is this correct? right...,squirrel hill of fifth avenue
4,4,2061123000,going to fifth avenue . is this correct?,yes


In [4]:
len(t2)

7760

In [5]:
t2.loc[t2.cust_text.isna()==True,"cust_text"]="None"

Given this data is about bus schedule customers would often use bus stop and place names. We took bus stop names of Pittsburg and added few more names to the list like - downtown, malls, college etc. We would use this file to replace some of the bus stop names with some common name to help in genearlization

In [6]:
import numpy as np
bus_sch = np.load("bus_stops.npz",allow_pickle=True) #make sure you use the .npz!
bus_sch1 = list(bus_sch['arr_0'])

In [23]:
bus_sch1

['freeporroad',
 'springarden',
 'bellevue',
 'shadeland',
 'coraopolis',
 'fairywood',
 'banksville',
 'bowehill',
 'carrick',
 'homesteaparlimited',
 'hazelwood',
 'nortbraddock',
 'lawrenceville-waterfront',
 'edgewootowcenter',
 'hamilton',
 'eashills',
 'liberty',
 'butlestreet',
 'bridgevillflyer',
 'easbusway-alstops',
 'penhillflyer',
 'monroevillflyer',
 'swissvalflyer',
 'baldwimanoflyer',
 'mounroyal',
 'perrysville',
 'ohivalley',
 'manchester',
 'mccoy',
 'airporflyer',
 'greetree',
 'bailey',
 'carriclimited',
 'nortside-oakland-soutside',
 'greenfield',
 'braddock-swissvale',
 'squirrehill',
 'negley',
 'homewood-squirrehill',
 'oahill',
 'friendship',
 'lawrenceville-hazelwood',
 'rosflyer',
 'alleghenvalleflyer',
 'lincolparflyer',
 'braddochillflyer',
 'lincolhighwaflyer',
 'elizabetflyer',
 'trohill',
 'fineview',
 'charles',
 'emswortlimited',
 'wespark',
 'robinson',
 'brookline',
 'knoxville',
 'homevilllimited',
 'glassport',
 'movalley',
 'mckeesport-homestead',

Replacing some common sentences in the bot_text

In [24]:
string_repl = "welcome to the cmu let's go bus information system.  to get help at any time, just say help or press zero. what can i do for you?  i am an automated spoken dialogue system that can give you schedule information for bus routes in pittsburgh's east end. you can ask me about the following buses: 28x, 54c, 56u, 59u, 61a, 61b, 61c, 61d, 61f, 64a, 69a, and 501. what bus schedule information are you looking for? for example, you can say, when is the next 28x from downtown to the airport? or i'd like to go from mckeesport to homestead tomorrow at 10 a.m."
t2["bot_text"] = t2["bot_text"].str.replace("welcome.*mckeesport to homestead.*automated spoken dialogue.*","greeting_templ")
t2["trim_bot_txt"] = t2["bot_text"].str.replace('goodbye.*','goodbye')


Now we will replace the place names in cust_text and bot_text using the bus_sch1 list with the function in Code 19 -  repl_place_bus using “fuzz.ratio” between words in the sentences and bus_sch1.”fuzz.ratio” calculates the similarity between strings using Levenshtein Distance (minimum number of single character edits required to change one word to another). “fuzzywuzzy” is the package used to calculate “fuzz.ratio”. You can “pip” install “fuzzywuzzy”


In [25]:
from fuzzywuzzy import fuzz 
str_list = []
def repl_place_bus(row,req_col):
    #for i,row in t2.iterrows():
    ctext = row[req_col]
    fg=0
    str1=ctext

    try:
        for j in ctext.split():

            for k in bus_sch1:
                score = fuzz.ratio(j,k )
                if(score>=70):
                    #print (i,j,k)
                    fg=1
                    break;
            #if(fg==1):
                #break;
            if(fg==1):
                fg=0
                str1 = str1.replace(j," place_name ")

    except:
        print (j,i)
    #str_list.append(str1)
        
    return str1

We have 7000+ rows. In each row we are iterating through the words in the sentences. This would increase the run-time very much. Inorder to run it faster, we will be applying the functions on parallelization in python. We would run the function simultanelusly across multiple “processes” using a library called “joblib”. The “n_jobs” parameter provides the number of processes that are going to be run in parallel.

Applying the function for customer text

In [26]:
from joblib import Parallel, delayed
com0 = Parallel(n_jobs=12, backend="threading",verbose=1)(delayed(repl_place_bus)(row,"cust_text") for i,row in t2.iterrows())

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    2.6s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:    4.9s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:    8.2s
[Parallel(n_jobs=12)]: Done 1226 tasks      | elapsed:   11.5s
[Parallel(n_jobs=12)]: Done 1776 tasks      | elapsed:   17.1s
[Parallel(n_jobs=12)]: Done 2426 tasks      | elapsed:   22.6s
[Parallel(n_jobs=12)]: Done 3176 tasks      | elapsed:   29.2s
[Parallel(n_jobs=12)]: Done 4026 tasks      | elapsed:   37.8s
[Parallel(n_jobs=12)]: Done 4976 tasks      | elapsed:   46.2s
[Parallel(n_jobs=12)]: Done 6026 tasks      | elapsed:   55.6s
[Parallel(n_jobs=12)]: Done 7176 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 7760 out of 7760 | elapsed:  1.3min finished


Applying the function for bot text

In [27]:
bot_list = Parallel(n_jobs=12, backend="threading",verbose=1)(delayed(repl_place_bus)(row,"trim_bot_txt") for i,row in t2.iterrows())

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    1.9s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:   14.8s
[Parallel(n_jobs=12)]: Done 426 tasks      | elapsed:   43.1s
[Parallel(n_jobs=12)]: Done 776 tasks      | elapsed:  1.2min
[Parallel(n_jobs=12)]: Done 1226 tasks      | elapsed:  1.7min
[Parallel(n_jobs=12)]: Done 1776 tasks      | elapsed:  2.3min
[Parallel(n_jobs=12)]: Done 2426 tasks      | elapsed:  3.1min
[Parallel(n_jobs=12)]: Done 3176 tasks      | elapsed:  3.9min
[Parallel(n_jobs=12)]: Done 4026 tasks      | elapsed:  5.0min
[Parallel(n_jobs=12)]: Done 4976 tasks      | elapsed:  6.2min
[Parallel(n_jobs=12)]: Done 6026 tasks      | elapsed:  7.5min
[Parallel(n_jobs=12)]: Done 7176 tasks      | elapsed:  8.9min
[Parallel(n_jobs=12)]: Done 7760 out of 7760 | elapsed:  9.6min finished


In [28]:
len(com0),len(t2),len(bot_list)

(7760, 7760, 7760)

In [29]:
t2.head()

,Unnamed: 0,user_id,bot_text,cust_text,trim_bot_txt
0,0,2061123000,welcome to the cmu let's go bus information s...,penn at butler time is it is the centre at hig...,welcome to the cmu let's go bus information s...
1,1,2061123000,leaving from oakland . is this correct?,oakland,leaving from oakland . is this correct?
2,2,2061123000,leaving from oakland . is this correct?,yes,leaving from oakland . is this correct?
3,3,2061123000,leaving from oakland . is this correct? right...,squirrel hill of fifth avenue,leaving from oakland . is this correct? right...
4,4,2061123000,going to fifth avenue . is this correct?,yes,going to fifth avenue . is this correct?


In [30]:
t2["corrected_cust"] = com0
t2["corrected_bot"] = bot_list

The columns “corrected_cust” and “corrected_bot” contains the replaced sentences. We will now do the following preprocesses - directions (since its a bus booking), normalizing am/pm, replacing number words to numbers, bus names (like 8a,11c etc), time, replacing numbers

In [31]:
direct_list = ['east','west','north','south']

In [32]:
def am_pm_direct_repl(t2,col):
    t2[col] = t2[col].str.replace('p.m','pm')
    t2[col] = t2[col].str.replace('p m','pm')
    t2[col] = t2[col].str.replace('a.m','am')
    t2[col] = t2[col].str.replace('a m','am')
    
    for i in direct_list:
        t2[col] = t2[col].str.replace(i,' direction ')
   
    return t2

In [34]:
t3 =  am_pm_direct_repl(t2,"corrected_cust")
t3 =  am_pm_direct_repl(t2,"corrected_bot")

The following function using “num2words” package gets numeric numbers for number-words like “one”, “two”, “three” etc. We then use this to replace our sentences with the right numbers. We would be replacing “th” words. For eg -fifth avenue gets replaced with “num_th”.

In [35]:
from num2words import num2words
word_num_list = []
word_num_th_list = []
for i in range(0,100):
    word_num_list.append(num2words(i))
    word_num_th_list.append(num2words(i,to='ordinal'))

print (word_num_list[0:5])
print (word_num_th_list[0:5])

['zero', 'one', 'two', 'three', 'four']
['zeroth', 'first', 'second', 'third', 'fourth']


In [36]:
def repl_num_words(t2,col):
    for num,i in enumerate(word_num_th_list):
        t2[col] = t2[col].str.replace(i,"num_th")
        t2[col] = t2[col].str.replace(word_num_list[num],str(num))
    return t2

In [37]:
t4 =  repl_num_words(t3,"corrected_cust")
t4 =  repl_num_words(t3,"corrected_bot")

Replacing bus and time names herewith

In [38]:
t4["corrected_cust"] = t4["corrected_cust"].str.replace('[0-9]{1,2}[a-z]{1,2}','bus_name')
t4["corrected_cust"] = t4["corrected_cust"].str.replace('[0-9]{1,2}[\.\s]*[0-9]{1,2}[\s]*pm','time_name')
t4["corrected_cust"] = t4["corrected_cust"].str.replace('[0-9]{1,2}[\.\s]*[0-9]{1,2}[\s]*am','time_name')

t4["corrected_bot"] = t4["corrected_bot"].str.replace('[0-9]{1,2}[a-z]{1,2}','bus_name')
t4["corrected_bot"] = t4["corrected_bot"].str.replace('[0-9]{1,2}[\.\s]*[0-9]{0,2}[\s]*pm','time_name')
t4["corrected_bot"] = t4["corrected_bot"].str.replace('[0-9]{1,2}[\.\s]*[0-9]{0,2}[\s]*am','time_name')

Next  we replace special characters and split the bot sentences with a sentence separator “sep_sent”. This will help us to “templatize” each of the bot sentences and generate these templates from the decoder  later

In [39]:
t4["corrected_bot1"] = t4["corrected_bot"].str.replace('\xa0','')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('[^a-z0-9\s\.\?]+','')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('[0-9]{1,3}',' short_num ')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('[0-9]{4,}',' long_num ')

t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('(\.\s){1,3}',' sep_sent ')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('(\.){1,3}',' sep_sent ')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('\?',' sep_sent ')
t4["corrected_bot1"] = t4["corrected_bot1"].str.replace('(sep_sent )+',' sep_sent ')



Replacing special characters in customer sentences. This is done in order to decrease the sparsity of the data. For example  - consider 2 sentences - “is this correct...” and “is this correct”. The word “correct” is same except in both cases – except one has 3 dots and the other has none. When we tokenize we want to make sure that these 2 words are treated the same 

In [40]:
t4["corrected_cust"] = t4["corrected_cust"].str.replace('(\.\s){1,3}',' ')
t4["corrected_cust"] = t4["corrected_cust"].str.replace('(\.){1,3}',' ')

In [41]:
t4["corrected_cust"].to_clipboard()

C:\Users\user\Anaconda2\envs\py36\lib\site-packages\pandas\io\clipboards.py:127: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  obj.to_csv(buf, sep=sep, encoding='utf-8', **kwargs)


In [42]:

len(t3["corrected_bot"].value_counts())

2447

Since we have taken a corpus which is an interaction between a bot and customer, some of the sentences are repetitive across different conversations. Given the nature of customer service data even if it had been a human to human conversation, there would be a good amount of repetitive sentences used across different dialogues. These repetitive sentences can be converted to “templates”. 

In [43]:
templ = pd.DataFrame(t4["corrected_bot1"].value_counts()).reset_index()
templ.columns = ["sents","count"]

In [44]:
templ.head()

,sents,count
0,welcome to the cmu lets go bus information sys...,315
1,you can say when is the next bus when is the p...,244
2,for example you can say when is the next busna...,176
3,welcome to the cmu lets go bus information sy...,175
4,leaving at timename sep_sent did i get that ...,162


Getting all the unique sentences into a list. Remaking the list into a dataframe to assign each sentence to a template id

In [45]:
df = templ.sents.str.split("sep_sent",expand=True)
sents_all = []
for i in df.columns:
    l1 = list(df[i].unique())
    sents_all = sents_all + l1
sents_all1 = set(sents_all)

sents_all_df = pd.DataFrame(sents_all1)
sents_all_df.columns = ["sent"]
sents_all_df["ind"] = sents_all_df.index
sents_all_df["ind"] = "templ_" + sents_all_df["ind"].astype('str')

In [46]:
sents_all_df.head()

,sent,ind
0,and arrives at numth placename at grant at...,templ_0
1,if you placename to leave from wilkinsburg ...,templ_1
2,for example you can say short_num short_num ...,templ_2
3,leaving from cmu,templ_3
4,and arrives at numth placename at grant at ...,templ_4


In [47]:
sents_all_df.to_clipboard()

In [48]:
sents_all_df["cnt"]  = sents_all_df["sent"].str.count(" ")

In [49]:
sents_all_df["cnt"].describe()

count    1503.000000
mean       14.242848
std         7.610017
min         0.000000
25%         8.000000
50%        13.000000
75%        19.000000
max        47.000000
Name: cnt, dtype: float64

Now we take our original dataset of t4 and assign different sentences to the template ids. First we create a dictionary with sentences as the keys and template ids

In [50]:
sent_all_df1 = sents_all_df.loc[:,["sent","ind"]]
df_sents = sent_all_df1.set_index('sent').T.to_dict('list')

In [51]:
t4["corrected_bots_sents"] = t4["corrected_bot1"].str.split("sep_sent")

index_list=[]
for i, row in t4.iterrows():
    sent_list = row["corrected_bots_sents"]
    str_index = ""
    for j in sent_list:
        if(len(j)>=3):
            str_index = str_index + " "  + str(df_sents[j][0])
    index_list.append(str_index)
    

In [52]:
t4["bots_templ_list"] = index_list

In [55]:
t4.tail()

,Unnamed: 0,user_id,bot_text,cust_text,trim_bot_txt,corrected_cust,corrected_bot,corrected_bot1,corrected_bots_sents,bots_templ_list,u_id_shift,corrected_bot1_shift,bots_templ_list_shift
7755,7755,2070619029,leaving from homestead . did i get that righ...,far,leaving from homestead . did i get that righ...,far,leaving from place_name . did i get that ...,leaving from placename sep_sent did i get...,"[ leaving from placename , did i get that...",templ_535 templ_1206,2.070619e+09,for example you can say placename placename...,templ_1029
7756,7756,2070619029,"for example, you can say, mckeesport, murray ...",mckeesport,"for example, you can say, mckeesport, murray ...",place_name,"for example, you can say, place_name place...",for example you can say placename placename...,[for example you can say placename placenam...,templ_1029,2.070619e+09,going to placename sep_sent did i get tha...,templ_868 templ_1343
7757,7757,2070619029,going to mckeesport . did i get that right?,yes,going to mckeesport . did i get that right?,yes,going to place_name . did i get that plac...,going to placename sep_sent did i get tha...,"[ going to placename , did i get that pl...",templ_868 templ_1343,2.070619e+09,going to placename sep_sent did i get tha...,templ_868 templ_745
7758,7758,2070619029,going to mckeesport . did i get that right? ...,next bus,going to mckeesport . did i get that right? ...,next bus,going to place_name . did i get that plac...,going to placename sep_sent did i get tha...,"[ going to placename , did i get that pl...",templ_868 templ_745,2.070619e+09,i think you placename the next bus sep_se...,templ_965 templ_78
7759,7759,2070619029,i think you want the next bus . am i right?,yes,i think you want the next bus . am i right?,yes,i think you place_name the next bus . am i...,i think you placename the next bus sep_se...,"[ i think you placename the next bus , am...",templ_965 templ_78,NaN,end_of_chat,end_of_chat


Our training set is like bot asks a question and the customer says the answer. But for training our input would be customer sentence and the response from the bot as the output. So we will shift our “corrected_bot1” to one sentence after. That way the corpus becomes incoming customer text and the right output from the bot. We are shifting the user_id to mark the end of  the chat

In [56]:
t4["u_id_shift"] = t4["user_id"].shift(-1)
t4["corrected_bot1_shift"] = t4["corrected_bot1"].shift(-1)
t4["bots_templ_list_shift"] = t4["bots_templ_list"].shift(-1)

t4.loc[t4.u_id_shift!=t3.user_id,"corrected_bot1_shift"] = "end_of_chat"
t4.loc[t4.u_id_shift!=t3.user_id,"bots_templ_list_shift"] = "end_of_chat"

req_cols = ["user_id","corrected_cust","corrected_bot1_shift","bots_templ_list_shift"]
t5 = t4[req_cols]

t5.tail()

,user_id,corrected_cust,corrected_bot1_shift,bots_templ_list_shift
7755,2070619029,far,for example you can say placename placename...,templ_1029
7756,2070619029,place_name,going to placename sep_sent did i get tha...,templ_868 templ_1343
7757,2070619029,yes,going to placename sep_sent did i get tha...,templ_868 templ_745
7758,2070619029,next bus,i think you placename the next bus sep_se...,templ_965 templ_78
7759,2070619029,yes,end_of_chat,end_of_chat


Save the processed dataset

In [68]:
t5.to_csv("lets_go_model_set1.csv")

We are now pickling the output of dictionary “df_sents” for mapping the templates to original sentences. In the encoder we would use “template_ids” for output. Then we remap using the dictionary to prin the actual sentences

In [54]:
import pickle
output = open('dict_templ.pkl', 'wb')

# Pickle dictionary using protocol 0.
pickle.dump(df_sents, output)